# Generate predictions for WN18
Generate predictions for WN18 and store the as a collated dataframe for a set of given predictions.

In [1]:
import pykeen
import pykeen.datasets
import pykeen.models
import pykeen.predict
import pandas as pd
import numpy as np
import pickle
import os
import polars as pl
import torch

## Load in each model

### Load dataset

In [2]:
dataset = pykeen.datasets.get_dataset(dataset="WN18")

/home/rogertu/miniforge3/envs/mini_semmed/lib/python3.12/site-packages/pykeen/triples/triples_factory.py:763: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = dict(torch

### Load TransE model

In [3]:
# load transe model
transe_model = pykeen.models.TransE(
    triples_factory=dataset.training,
    embedding_dim=500,
    scoring_fct_norm=2,
    random_seed=2107241829,  # use same seed as training otherwise model and chkpt train/test split will be different
)

# load chkpt
transe_chkpt = torch.load(
    pykeen.constants.PYKEEN_CHECKPOINTS.joinpath(
        "TransE_WN18_2.pt"
    ),  # accidentally deleted original. rm is sometimes dangerous
)
# attach state to model
transe_model.load_state_dict(transe_chkpt["model_state_dict"])

/tmp/ipykernel_228982/3608602754.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transe_chkpt = torch.load(


<All keys matched successfully>

### Load RotatE model

In [5]:
# load rotate model
rotate_model = pykeen.models.RotatE(
    triples_factory=dataset.training,
    embedding_dim=250,  # Note this is half the size of the actual embedding dim listed because rotate doubles the embedding dim
    random_seed=2382073919,  # use same seed as training otherwise model and chkpt train/test split will be different
)

# load chkpt
rotate_chkpt = torch.load(
    pykeen.constants.PYKEEN_CHECKPOINTS.joinpath("RotatE_WN18.pt"),
)
# attach state to model
rotate_model.load_state_dict(transe_chkpt["model_state_dict"])

/tmp/ipykernel_228982/165234142.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rotate_chkpt = torch.load(


<All keys matched successfully>

### Load ComplEx model

In [6]:
# load ComplEx model
complex_model = pykeen.models.ComplEx(
    triples_factory=dataset.training,
    embedding_dim=500,
    random_seed=2152372825,  # use same seed as training otherwise model and chkpt train/test split will be different
    regularizer_kwargs=dict(weight=0.00001, p=3),
)

# load chkpt
complex_chkpt = torch.load(
    pykeen.constants.PYKEEN_CHECKPOINTS.joinpath("ComplEx_WN18.pt"),
)
# attach state to model
complex_model.load_state_dict(complex_chkpt["model_state_dict"])

/tmp/ipykernel_228982/2070738479.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  complex_chkpt = torch.load(


<All keys matched successfully>

### Load DistMult model

In [7]:
# load distmult model
distmult_model = pykeen.models.DistMult(
    triples_factory=dataset.training,
    embedding_dim=1000,
    random_seed=1191075500,  # use same seed as training otherwise model and chkpt train/test split will be different
    regularizer_kwargs=dict(weight=0.00001, p=3),
)

# load chkpt
distmult_chkpt = torch.load(
    pykeen.constants.PYKEEN_CHECKPOINTS.joinpath("DistMult_WN18.pt"),
)
# attach state to model
distmult_model.load_state_dict(distmult_chkpt["model_state_dict"])

/tmp/ipykernel_228982/2654902952.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  distmult_chkpt = torch.load(


<All keys matched successfully>

## Evaluate model on a fixed test set

### how many nodes and relations are there that we can sample against?

In [8]:
f"Number of Nodes: {dataset.num_entities:,}"

'Number of Nodes: 40,943'

In [9]:
f"Number of Relations: {dataset.num_relations:,}"

'Number of Relations: 18'

In [10]:
# dataframe of testing triples and their correct answers
dataset.testing.tensor_to_df(dataset.testing.mapped_triples)

,head_id,head_label,relation_id,relation_label,tail_id,tail_label
0,13,3826,3,_hypernym,0,1740
1,17,4475,3,_hypernym,16,4258
2,24,6238,3,_hypernym,618,104868
3,29,6802,3,_hypernym,30,7012
4,31,7328,1,_derivationally_related_form,32527,10803193
...,...,...,...,...,...,...
4995,40901,15275598,3,_hypernym,40892,15272029
4996,40904,15278281,3,_hypernym,40917,15286249
4997,40907,15279596,12,_part_of,40908,15279957
4998,40916,15286042,12,_part_of,2964,543233


### Get test triples (at least a small set of it)

In [11]:
# how the tensor looks
dataset.testing.mapped_triples

tensor([[   13,     3,     0],
        [   17,     3,    16],
        [   24,     3,   618],
        ...,
        [40907,    12, 40908],
        [40916,    12,  2964],
        [40936,    15, 22643]])

In [12]:
# size of the tensor
dataset.testing.mapped_triples.shape

torch.Size([5000, 3])

In [13]:
# get 1000 random indices to slice the testing triples
random_ind = np.random.choice(
    np.array(range(0, dataset.testing.mapped_triples.shape[0])), 1000, replace=False
)

In [14]:
# sliced tensor of testing triples
dataset.testing.mapped_triples[random_ind].shape

torch.Size([1000, 3])

In [15]:
# list of lists of triples
# [[head, relation, tail], ...]
test_set = dataset.testing.mapped_triples[random_ind].tolist()

### get top 1000 predictions in our random sample.
* May or may not be exactly 1000 unique entity/relation combinations

In [16]:
# write afunction to make predictions on the models
def get_top_tail_predictions(model, test_set, dataset, k=None) -> pd.DataFrame:
    """
    Given a model, and a test set, return the top predictions for the test set

    model: pykeen.models.Model
    test_set: list of lists of triples [[head, relation, tail], ...]
    dataset: pykeen.datasets.dataset.TriplesFactory
    k: int, number of top predictions to return

    returns: pd.DataFrame
    """
    # create dictionaries for entities and relations
    id2entity = {v: k for k, v in dataset.entity_to_id.items()}
    id2relation = {v: k for k, v in dataset.relation_to_id.items()}

    # get the top predictions for the first entry in the test set
    res_ls = []
    for i in test_set:
        # generate predictions and cast to a polars dataframe
        polars_df = pl.DataFrame(
            pykeen.predict.predict_target(
                model=model, triples_factory=dataset, head=i[0], relation=i[1]
            ).df
        )
        # sort the predictions by score, add head and relation ids
        polars_df = polars_df.with_columns(
            pl.col("score").sort(descending=True),
            head_id=i[0],  # assign head_id
            rel_id=i[1],  # assign relation_id
        )
        res_ls.append(polars_df)

    # rename entities in head/tail/relation from ids to actual names
    # collapse tail_ids to a single row based on head and relation_ids
    res_df = (
        pl.concat(res_ls)
        .with_columns(
            # rename entities in head/tail/relation from ids to actual names
            pl.col("tail_id").cast(pl.String).replace(id2entity),
            pl.col("head_id").cast(pl.String).replace(id2entity),
            pl.col("rel_id").cast(pl.String).replace(id2relation),
        )
        .unique(["head_id", "rel_id", "tail_id"])
        .group_by(["head_id", "rel_id"])
        .agg("tail_id", maintain_order=True)
    )

    # return top k predictions
    if k > 0:
        res_df = res_df.with_columns(pl.col("tail_id").list.head(k))

    return res_df

### make predictions and export

In [17]:
transe_df = get_top_tail_predictions(transe_model, test_set, dataset, k=1000)

In [18]:
distumult_df = get_top_tail_predictions(distmult_model, test_set, dataset, k=1000)

In [19]:
complex_df = get_top_tail_predictions(complex_model, test_set, dataset, k=1000)

In [20]:
rotate_df = get_top_tail_predictions(rotate_model, test_set, dataset, k=1000)

#### add column name identifier to each dataframe.
* then stack them!

In [21]:
transe_df = transe_df.with_columns(model=pl.lit("TransE"))
distumult_df = distumult_df.with_columns(model=pl.lit("DistMult"))
complex_df = complex_df.with_columns(model=pl.lit("ComplEx"))
rotate_df = rotate_df.with_columns(model=pl.lit("RotatE"))

# combine the results
combined_df = pl.concat([transe_df, distumult_df, complex_df, rotate_df])

In [22]:
combined_df.head()

head_id,rel_id,tail_id,maintain_order,model
str,str,list[str],bool,str
"""12102133""","""_hyponym""","[""659349"", ""1159964"", … ""5289057""]",true,"""TransE"""
"""12508077""","""_hypernym""","[""5833840"", ""2441022"", … ""10278128""]",true,"""TransE"""
"""2950482""","""_hypernym""","[""2016523"", ""9815790"", … ""12505563""]",true,"""TransE"""
"""10271216""","""_derivationall…","[""1344293"", ""1500082"", … ""8304895""]",true,"""TransE"""
"""367976""","""_hyponym""","[""5722208"", ""680485"", … ""1902783""]",true,"""TransE"""


In [23]:
combined_df.shape

(3880, 5)

In [24]:
assert (
    combined_df.shape[0] / 4 == combined_df.unique(["head_id", "rel_id"]).shape[0]
), "Some predictions are not made between all algorithms"

#### export the sample list as well as the parquet

In [25]:
with open(
    "/home/rogertu/projects/semmed/semmed/data/benchmark_data/WN18_1000_sampled_test.pkl",
    "wb",
) as f:
    pickle.dump(combined_df, f)

In [26]:
combined_df.write_parquet(
    "/home/rogertu/projects/semmed/semmed/data/benchmark_data/WN18_1000_sampled_test_predictions.parquet"
)